<div align="center">
    <img src="images/um_logo.png" alt="image">
</div>

# Computación II


# ***Procesos II***

## Entendiendo las Tablas de Procesos y los Procesos Zombies
Linux es un sistema operativo multitarea y multiusuario que permite ejecutar simultáneamente múltiples procesos. Un proceso, en términos simples, es una instancia de un programa en ejecución. Cada proceso en Linux se identifica por un PID (Identificador de Proceso) único. Los procesos pueden crear otros procesos a través de una combinación de llamadas al sistema `fork()` y `exec()`.

`fork()` y `exec()` son dos llamadas al sistema en Unix y sistemas operativos basados en Unix, como Linux, que están estrechamente relacionadas con la creación y gestión de procesos. Aunque a menudo se usan juntas, tienen propósitos muy diferentes.

### `fork()`
La llamada al sistema `fork()` se utiliza para crear un nuevo proceso, conocido como proceso hijo, que es una copia casi exacta del proceso que lo llamó, conocido como proceso padre. El proceso hijo recibe una copia de los datos, el código y el espacio de pila del proceso padre, pero tiene su propio identificador de proceso (PID). Sin embargo, hay algunas diferencias entre el proceso hijo y el padre, como los valores de retorno de `fork()` y los identificadores de proceso.

Después de `fork()`, ambos procesos, el padre y el hijo, continúan ejecutándose desde el punto donde `fork()` fue llamado, pero con diferentes espacios de memoria. Esto significa que los cambios realizados en la memoria por el proceso padre o el proceso hijo después de `fork()` no se reflejan en el otro proceso.

In [ ]:
import os

def main():
    print("Inicio del programa (proceso padre).")

    pid = os.fork()

    if pid == 0:
        # Este código se ejecuta en el proceso hijo
        print("Hola, soy el proceso hijo.")
    else:
        # Este código se ejecuta en el proceso padre
        print(f"Hola, soy el proceso padre, y mi hijo tiene PID {pid}.")

    print("Fin del programa.")

if __name__ == "__main__":
    main()


### `exec()`
La familia de funciones `exec()` se utiliza para ejecutar un nuevo programa en el espacio de direcciones de un proceso, reemplazando el programa actual. Después de que `exec()` se ejecuta con éxito, el proceso anterior desaparece y se carga un nuevo programa en memoria para ejecutar. Esto se utiliza típicamente después de un `fork()` para ejecutar un nuevo programa en el proceso hijo mientras el proceso padre puede realizar otras tareas o esperar a que el proceso hijo termine.

`exec()` viene en varias variantes como `execl()`, `execp()`, `execv()`, etc., que difieren en cómo reciben argumentos, pero todas reemplazan el proceso actual con un nuevo programa especificado.

In [ ]:
import os

def main():
    print("Ejecutando 'ls' para listar directorios...")

    # Reemplaza el proceso actual con 'ls'
    # Nota: Este código no se ejecutará más allá de este punto en el proceso actual
    os.execlp('ls', 'ls', '-l')

    # Esta línea no se ejecutará
    print("Esta línea no se mostrará.")

if __name__ == "__main__":
    main()

## Procesos Zombies
Un proceso zombi es un proceso que ha completado su ejecución pero aún tiene una entrada en la tabla de procesos, esperando que su proceso padre lea su estado de salida. Esto ocurre cuando el proceso padre no ejecuta una llamada al sistema wait() para recoger el estado de terminación de su proceso hijo. Los procesos zombis consumen un mínimo de recursos, pero dejar muchos procesos zombis sin gestionar puede eventualmente llenar la tabla de procesos.

Para identificar procesos zombis, se puede utilizar:

### Manejando Procesos Zombies en Python
Python puede crear y gestionar procesos utilizando el módulo subprocess. Aquí hay un ejemplo de cómo crear un proceso zombi y luego gestionarlo:

In [ ]:
import subprocess
import os
import time

# Crear un proceso hijo que termina inmediatamente
pid = os.fork()
if pid == 0:
    # Proceso hijo
    print("Este es el proceso hijo, terminando ahora.")
    os._exit(0)
else:
    # Proceso padre
    print(f"Este es el proceso padre, dejando al hijo {pid} como zombi.")
    time.sleep(20)  # Simular trabajo para dejar al hijo como zombi
    print("El proceso padre ha terminado, el hijo debería ser adoptado por init.")

# Nota: Este código debe ser ejecutado en un entorno seguro, ya que crea un proceso zombi.


En este ejemplo, el proceso hijo termina inmediatamente después de su creación, convirtiéndose en un zombi hasta que el proceso padre termina su ejecución. Después de esto, el proceso zombi es adoptado por el proceso init, que automáticamente llamará a wait() para eliminar el zombi de la tabla de procesos.

## Analisis de código
Este es un ejemplo de código secuencial en Python que demuestra cómo se ejecutan las instrucciones una tras otra, sin la creación de procesos paralelos o concurrentes. 

In [ ]:
"""
Ejemplo de código secuencial.
Las instrucciones se van ejecutando una detrás de la otra

NOTA DE CLASE: mientras se ejecuta correr ps fax|grep python
"""


import time
import os

print('INICIO')
print('PID: %d  --  PPID: %d' % (os.getpid(), os.getppid()))

for i in range(5, 0, -1):
  print(i)
  time.sleep(1)


print('\nFIN')
print('PID: %d  --  PPID: %d' % (os.getpid(), os.getppid()))

El siguiente fragmento de código ilustra el uso de la función os.system en Python para ejecutar un script externo, denominado ejemplo_1.py, dos veces de manera secuencial. La característica clave a destacar aquí es que os.system es una función bloqueante. Esto significa que cada llamada a os.system debe completar su ejecución antes de que el bucle pueda continuar y comenzar la ejecución del comando siguiente.

En este contexto, el bucle for realiza dos iteraciones. En cada iteración, invoca el comando python ejemplo_1.py a través de os.system, ejecutando así el script externo. Dado que os.system espera a que el comando termine antes de regresar el control al script Python que lo llamó, el efecto es que el script ejemplo_1.py se ejecuta completamente una vez, y solo después de que termine, inicia la segunda ejecución.

Este comportamiento es especialmente relevante en contextos donde la secuencialidad es importante, por ejemplo, cuando el script llamado realiza alguna tarea que debe completarse antes de iniciar una nueva ejecución o cuando el resultado de la primera ejecución afecta a la siguiente.

In [ ]:
"""
Se puede ver como os.system es una función bloqueante. Por lo tanto debe esperar a que termine el primer proceso para comenzar el siguiente
"""

import os

for i in range(2):
  os.system("python ejemplo_1.py")

El siguiente fragmento de código demuestra el uso de Popen del módulo subprocess en Python, que permite la ejecución no bloqueante de procesos. A diferencia de os.system, Popen inicia un proceso y devuelve inmediatamente el control al script que lo llamó, permitiendo que el script continúe su ejecución sin esperar a que el proceso iniciado concluya.

El código ejecuta un bucle que itera dos veces, donde en cada iteración se inicia un nuevo proceso para ejecutar el script ejemplo_1.py utilizando Popen. Gracias a la naturaleza no bloqueante de Popen, ambas invocaciones del script externo pueden correr de manera concurrente, es decir, al mismo tiempo, pero sin necesariamente ejecutarse en paralelo a menos que el sistema disponga de múltiples núcleos de CPU y el sistema operativo decida ejecutar esos procesos en núcleos distintos.

Después de iniciar los procesos, el script espera un segundo mediante time.sleep(1) y luego imprime 'FIN DEL PROCESO PADRE'. Este mensaje se muestra antes de que los procesos iniciados con Popen necesariamente hayan terminado, ilustrando el comportamiento no bloqueante de Popen y cómo permite la concurrencia en la ejecución de tareas.

Es importante distinguir entre concurrente y paralelo: la concurrencia se refiere a la capacidad de progresar en múltiples tareas al mismo tiempo en el contexto de un programa, mientras que el paralelismo implica la ejecución simultánea de tareas en diferentes núcleos de procesador, ofreciendo una mejora real en el rendimiento mediante la ejecución literalmente al mismo tiempo.

In [ ]:
"""
Popen no es bloqueante, por lo que no va esperar a que termine un proceso para comenzar otro.
Procesos, tareas o hilos concurrentes: ocurren al mismo tiempo
Paralelos: procesos, tareas o hilos concurrentes que se ejecutan en dos núcleos distintos
"""

from subprocess import Popen
import time

for i in range(2):
  Popen(["python", "ejemplo_1.py"])


time.sleep(1)
print('FIN DEL PROCESO PADRE')

El siguiente código ilustra el uso de la función os.execlp para reemplazar el proceso actual del script Python por un nuevo proceso, ejecutando el script hijo.py. La característica principal de las funciones de la familia exec es que reemplazan completamente el programa en el proceso actual con un nuevo programa, lo que significa que el proceso mantiene el mismo PID (Identificador de Proceso), pero ejecuta un código diferente. En este caso, el binario de Python ejecutando el script actual es reemplazado por un nuevo binario de Python ejecutando hijo.py.

1. Antes de la llamada a os.execlp: El script imprime el PID del proceso actual, identificándose como el proceso padre. Este PID es único y es asignado por el sistema operativo.

2. Uso de os.execlp: Esta función busca el ejecutable especificado (en este caso, python) en el PATH del sistema y lo ejecuta, pasando ./hijo.py como argumento. Importante, reemplaza el binario actual del proceso (el script Python en ejecución) con el binario de Python que ejecuta hijo.py, efectivamente transformando el proceso actual en el proceso que ejecuta hijo.py.

3. Después de la llamada a os.execlp: Cualquier código escrito después de la llamada a os.execlp nunca se ejecuta, porque el espacio de memoria del proceso que contenía el script original ha sido reemplazado por el nuevo programa. Esto significa que la línea print('DA IGUAL LO QUE SE EJECUTE EN ESTE PUNTO PORQUE EXEC MODIFICA EL BINARIO ACTUAL') nunca se ejecutará.

In [ ]:
"""
El padre y el hijo tendrán el mismo PID ya que el hijo se ejecuta modificando el binario del padre.

"""


import os



print('SOY EL PADRE (PID: %d)' % os.getpid())
os.execlp('python', 'python', './hijo.py')

print('DA IGUAL LO QUE SE EJECUTE EN ESTE PUNTO PORQUE EXEC MODIFICA EL BINARIO ACTUAL')
  



In [ ]:
#Codigo  de hijo.py
import os

print('SOY EL HIJO INDEPENDIENTE (PID: %d -- PPID: %d)' % (os.getpid(), os.getppid()))

El siguiente programa demuestra cómo crear un proceso zombi utilizando la bifurcación (`fork()`) y la ejecución (`execlp()`) en un sistema operativo tipo Unix/Linux.


In [ ]:
import os, time

pid = os.fork()

if pid == 0:
  os.execlp("sleep", "sleep", "5")

time.sleep(10)  
print('Finalizando el padre')

El siguiente script demuestra el uso de os.fork() para crear un proceso hijo en Python y luego utiliza os.execlp para reemplazar el proceso hijo con otro programa. También muestra cómo el proceso padre puede esperar por la finalización del proceso hijo usando os.wait(). Aquí te doy un resumen de su funcionamiento:

1. Inicio: El script comienza imprimiendo el PID del proceso actual, que es el proceso padre, seguido de un mensaje que indica que se realizará un fork.

2. Forking: Utiliza os.fork() para intentar crear un proceso hijo. Si os.fork() tiene éxito, devuelve dos veces: en el proceso padre, devuelve el PID del proceso hijo, y en el proceso hijo, devuelve 0. Si falla, imprime un mensaje de error.

3. Comportamiento del Padre: Si el valor retornado por os.fork() es mayor que 0, el código está en el proceso padre. El proceso padre imprime su PID nuevamente, luego llama a os.wait(). os.wait() pausa la ejecución del proceso padre hasta que uno de sus procesos hijos termina, devolviendo el PID del hijo y su estado de salida. Después de esperar al hijo, el padre imprime el retorno de os.wait() (que contiene el PID del hijo y su estado de salida) y finaliza con un mensaje que indica el fin del proceso padre.

4. Comportamiento del Hijo: Si el valor retornado por os.fork() es 0, el código está en el proceso hijo. El hijo imprime su propio PID y el PID de su proceso padre (PPID), luego usa os.execlp para reemplazar su ejecución con otro programa (en este caso, el script hijo.py). Debido a os.execlp, cualquier línea de código después de esta llamada en el proceso hijo nunca se ejecuta, ya que el espacio de memoria del proceso hijo ahora contiene el nuevo programa.

5. Observación Importante: El script menciona el uso de ps lf para observar procesos y ps ps, probablemente queriendo indicar el uso de ps para inspeccionar los procesos y sus relaciones padre-hijo. Sin embargo, parece haber un pequeño error tipográfico en la explicación de cómo ver procesos padre con ps ps, que no es un comando válido. Para ver la jerarquía de procesos, se puede usar ps -f, ps -ef, o pstree en la terminal de Linux.

Este código ilustra importantes conceptos de programación de sistemas en Python, como la creación de procesos, la espera por la finalización de procesos hijos, y la sustitución del programa en ejecución de un proceso.

In [ ]:
"""
Se pueden ver los procesos con ps lf.
Para ver los procesos padre se pueden ver con ps ps
"""


import os
import sys
import time

print('SOY EL PADRE (PID: %d)' % os.getpid())
print('fork --------------------------------')
try:
  ret = os.fork()
except OSError:
  print('ERROR AL CREAR EL HIJO')
  


if ret > 0:
  print('SOY EL PADRE (PID: %d )' % os.getpid())
  # Cuando bash ejecuta algo como este programa, se queda esperando el exit status del hijo. Si el padre no espera (os.wait()) no se devolverá el prompt por que bash no busca el exist status del hijo.
  ret = os.wait()
  print(ret)
  print('FIN DEL PADRE')
  
elif ret == 0:
  print('SOY EL HIJO (PID: %d -- PPID: %d)' % (os.getpid(), os.getppid()))
  
#  time.sleep(2)
  os.execlp('python', 'python', './hijo.py')
  
  print('DA IGUAL LO QUE SE EJECUTE EN ESTE PUNTO PORQUE EXEC MODIFICA EL BINARIO ACTUAL')

El siguiente programa demuestra el concepto de forking en sistemas operativos tipo Unix/Linux, así como el manejo de archivos y el comportamiento de los descriptores de archivos durante y después del fork. Aquí te detallo su funcionamiento:

#### Descripción General
Al inicio, el script abre (o crea si no existe) un archivo llamado archivo.txt en modo de escritura y lectura (w+). Esto significa que cualquier contenido previo del archivo será borrado y el archivo estará listo para leer y escribir desde el principio.

Luego, realiza un fork para crear un proceso hijo.

##### Proceso Padre
- El proceso padre (determinado por if (pid):) imprime su PID y luego intenta leer el contenido del archivo. Sin embargo, dado que el read() se realiza inmediatamente después de un sleep() y sin un seek(0) previo que reposicione el cursor al inicio del archivo, no podrá leer lo que el hijo escribió a menos que se descomente el fd.seek(0).

- El código comentado que escribe en el archivo desde el padre y luego llama a flush() está destinado a demostrar cómo ambos procesos, padre e hijo, pueden modificar el mismo archivo, y cómo es necesario utilizar flush() para asegurar que lo escrito se refleje inmediatamente en el archivo, ya que de lo contrario podría quedarse en el buffer de salida.

##### Proceso Hijo
- El proceso hijo escribe una línea en el archivo y luego llama a flush() para asegurar que el cambio se guarde en el archivo antes de que el proceso termine con sys.exit(0). Esto demuestra cómo ambos procesos, debido a que comparten el descriptor de archivo, pueden escribir en el mismo archivo.
##### Comportamiento del Descriptor de Archivo
- Después del fork, tanto el proceso padre como el hijo tienen su propio descriptor de archivo (aunque apuntando al mismo archivo abierto), lo que significa que comparten la posición actual del archivo. Es importante notar que cualquier operación de escritura o lectura afectará la posición del archivo en ambos procesos, a menos que se maneje cuidadosamente, como reajustando la posición del archivo con seek() después de leer o escribir.

- El comentario final sobre el uso de time.sleep(60) y la inspección de los descriptores de archivos con sudo ls -l /proc/pid/fd es una sugerencia para explorar cómo los descriptores de archivos se comparten entre el proceso padre e hijo, mostrando que ambos procesos tienen acceso al mismo archivo, lo cual puede ser verificado observando los enlaces simbólicos en los directorios de los procesos dentro de /proc.

In [ ]:
import os
import sys
import time



def main():
    
  fd = open('./archivo.txt', 'w+')
  
  
  pid = os.fork()

  if (pid): # Este es el proceso padre
    print('PADRE (PID: %d)' % os.getpid())
    time.sleep(1)
#      fd.seek(0)
    print(fd.read())
    
#    fd.write('Esto es una línea del PADRE')
#    fd.flush() #Cuando se escribe en un archivo se escribe sobre un buffer. Para que el contenido del buffer sea escrito en el archivo se debe usar flush o cerrar el archivo o terminar el proceso correctamente
  
  else: # Proceso hijo
    print('HIJO (PID: %d)' % os.getpid())
    fd.write('Esto es una línea del HIJO')
    fd.flush()
    sys.exit(0)
      
      
#  time.sleep(60) # Los files descriptors se comparten. Por ese motivo ambos pueden ver lo que esta pasando en el archivo. sudo ls -l /proc/pid/fd

if __name__ == "__main__":
  main()
 


El siguiente programa ilustra el uso de `os.fork()` en Python para crear un proceso hijo y cómo la memoria se maneja entre el proceso padre y el proceso hijo.

1. Inicialización: Se declara una variable var con el valor inicial de 100. Luego, el proceso padre imprime su PID.

2. Forking: Se llama a `os.fork()`, lo cual duplica el proceso. En el proceso padre, `os.fork()` devuelve el PID del proceso hijo, mientras que en el proceso hijo devuelve 0.

3. Bucle de Ejecución:
- Proceso Padre: Si el PID es no cero (indicando que el código se está ejecutando en el proceso padre), entra en un bucle donde incrementa el valor de var por 1 en cada iteración, imprime el valor actualizado de var y su dirección de memoria (indicada por `id(var)`), y luego pausa la ejecución por 1 segundo.
- Proceso Hijo: Si el PID es cero (indicando que el código se está ejecutando en el proceso hijo), entra en un bucle similar pero, en lugar de incrementar, decrementa el valor de var por 1 en cada iteración, imprime el valor actualizado y su dirección de memoria, y pausa la ejecución por 1 segundo.
  
4. Independencia de Variables: Aunque var empieza con el mismo valor en ambos procesos justo después del `fork()`, los cambios que cada proceso hace a var son independientes el uno del otro. Esto se debe a la semántica de "copia-en-escritura" de `fork()`, que hace que cada proceso tenga su propia copia independiente de las variables después de modificarlas. Las direcciones de memoria impresas con `id(var)` serán diferentes entre el padre y el hijo después de la primera modificación, evidenciando esta separación.

5. Observación de la Concurrency: Este script también demuestra cómo los procesos padre e hijo pueden ejecutarse concurrentemente, con el sistema operativo alternando entre ellos. Las impresiones en la salida estándar reflejan cómo el padre y el hijo están ejecutando sus bucles concurrentemente, modificando y reportando el valor de var de manera independiente.

In [ ]:
import os
import sys
import time

var = 100
print("PADRE: Soy el proceso padre y mi pid es: %d" % os.getpid())

pid = os.fork()

for i in range(10):
  if (pid): # Este es el proceso padre
      var += 1
      print("PADRE var: %d  --- %d" % (var, id(var)))
      time.sleep(1)
  
  else: # Proceso hijo
      var -= 1
      print("HIJO var: %d  --- %d" % (var, id(var)))
      time.sleep(1)

El siguiente programa realiza dos llamadas secuenciales a os.fork(), creando un total de cuatro procesos, incluyendo el proceso original. No es una "fork bomb" en el sentido tradicional, sino un ejemplo controlado de cómo se pueden crear múltiples procesos mediante el uso de fork().

##### Descripción del Proceso de Forking
1. Inicio del Script: El script comienza su ejecución en el proceso original (llamémosle A). En este punto, solo hay un proceso ejecutando el script.

2. Primera llamada a os.fork(): Esta línea duplica el proceso. Ahora tenemos dos procesos:

- El proceso original (A) continúa la ejecución del script.
- Un nuevo proceso hijo (B) es creado, que también comienza a ejecutar el script desde el punto justo después de la llamada a fork().
 
3. Segunda llamada a os.fork(): Tanto A como B ejecutan esta línea, y cada uno crea su propio proceso hijo. Esto resulta en cuatro procesos en total:

- El proceso original (A).
- El primer hijo (B), creado por la primera llamada a fork().
- Un hijo de A (llamémosle C), creado por la segunda llamada a fork() en A.
- Un hijo de B (llamémosle D), creado por la segunda llamada a fork() en B.

  

In [ ]:
import os
import time

def main():
    os.fork()
    os.fork()
    print('Hola mundo')
    
#    time.sleep(120) #Para ver la gerarqía  o el while
#    while True:
#      pass

if __name__ == "__main__":
  main()

# Flujos Estándar en Sistemas Unix-like

Los sistemas operativos tipo Unix, incluyendo Linux y macOS, manejan tres flujos estándar de datos que son utilizados por los programas. Estos flujos son:

1. **Entrada Estándar (stdin)**: Es el flujo de datos de entrada para un programa. Por defecto, está conectado al teclado, permitiendo que un programa lea datos ingresados por el usuario. Se puede redirigir para leer datos de un archivo o de otro programa.
   - **Identificador de Archivo**: 0
   - **Uso en Código**: Se accede a través de `sys.stdin` en Python.

2. **Salida Estándar (stdout)**: Es el flujo principal de datos de salida de un programa. Por defecto, se muestra en la terminal (consola), permitiendo que el programa envíe datos hacia el exterior, por ejemplo, imprimir resultados o mensajes para el usuario. Se puede redirigir para escribir a un archivo o a otro programa.
   - **Identificador de Archivo**: 1
   - **Uso en Código**: Se accede a través de `sys.stdout` en Python.

3. **Error Estándar (stderr)**: Es un flujo de datos de salida separado utilizado para enviar mensajes de error o de diagnóstico. Por defecto, también se muestra en la terminal, pero se puede redirigir de forma independiente a `stdout`, permitiendo separar los mensajes de error de la salida normal del programa.
   - **Identificador de Archivo**: 2
   - **Uso en Código**: Se accede a través de `sys.stderr` en Python.

## Ejemplo de Redirección en la Terminal

Para redirigir la salida estándar de un programa a un archivo, se puede utilizar el operador `>` en la terminal:


Para redirigir el error estándar a un archivo diferente:

Y para redirigir tanto la salida estándar como el error estándar al mismo archivo:

El siguiente programa demuestra cómo redirigir el flujo de error estándar (stderr) a un archivo para capturar mensajes de error.

In [ ]:
import sys, os


print('PID: %d' %os.getpid())


fh = open('test.txt', 'w')


input('Antes de redireccionar')
sys.stderr = fh
print('Esta línea va a test.txt', file=sys.stderr)

sys.Popen('ccc')

input('Despues de redireccionar')


sys.stderr = sys.__stderr__

fh.close()

Otra versión comentada y con manejo de errores:

In [ ]:
import sys, os

# Imprimir el PID del proceso actual
print('PID: %d' % os.getpid())

# Abrir (o crear) un archivo en modo de escritura
fh = open('test.txt', 'w')

# Pausa antes de la redirección, esperando entrada del usuario
input('Antes de redireccionar')

# Redirigir stderr al archivo abierto
sys.stderr = fh

# Escribir un mensaje en stderr, que ahora apunta a 'test.txt'
print('Esta línea va a test.txt', file=sys.stderr)

# Provocar intencionadamente un error para demostrar la redirección de stderr
try:
    sys.Popen('ccc')
except AttributeError as e:
    print(f"Error capturado: {e}", file=sys.stderr)

# Pausa después de la redirección, esperando entrada del usuario
input('Después de redireccionar')

# Restaurar stderr a su flujo estándar original
sys.stderr = sys.__stderr__

# Cerrar el archivo
fh.close()